In [130]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.metrics import make_scorer, roc_auc_score,confusion_matrix,f1_score,accuracy_score,precision_score,recall_score

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

#The model
from sklearn.ensemble import IsolationForest

#For preprocessing
from sklearn.preprocessing import MinMaxScaler,StandardScaler

# from tqdm import tqdm
from tqdm.notebook import tqdm
import warnings 
warnings.simplefilter("ignore")

### Preprocessing

In [94]:
data=pd.read_csv("/home/dah/anomalie_detection/anomalie_detection/data/creditcard.csv")


In [95]:

data=data.copy(deep=True)
data.drop_duplicates(keep="last",inplace=True)
target=data.Class
data=data.drop(["Class","Time"],axis=1)

In [96]:
scaler = MinMaxScaler()
data_scaled_array = scaler.fit_transform(data)

data = pd.DataFrame(data_scaled_array, columns=data.columns)

### Preparation des données d'entraînement et de test

In [97]:
xtrain,xtest,ytrain,ytest=train_test_split(data,target,test_size=0.15,random_state=42,shuffle=True)

### Définition du modèle

In [98]:
model=IsolationForest(contamination=0.17,bootstrap=False,n_estimators=500,n_jobs=-1,verbose=1)

### Entraînement du modèle

In [99]:
model.fit(xtrain,ytrain)

[Parallel(n_jobs=16)]: Using backend ThreadingBackend with 16 concurrent workers.
[Parallel(n_jobs=16)]: Done   2 out of  16 | elapsed:    0.9s remaining:    6.4s
[Parallel(n_jobs=16)]: Done  16 out of  16 | elapsed:    1.0s finished


IsolationForest(contamination=0.17, n_estimators=500, n_jobs=-1, verbose=1)

### Prédiction du modèle

In [100]:
ypred=model.predict(xtest)

### Transformation des targets
ypred return un vecteur contenant que de -1 pour des données anormales et 1 pour les données normales alors que les targets des <br>
des données initial contiennet des 0 pour des données normales et 1 pour des données anormales. Il faut donc transformer ce vecteur <br>
en un vecteur contenant des 1 pour des données normales et -1 pour les anomalies

In [101]:
def transform_target(target):
    n=target.shape[0]
    for i in range(n):
        if target.iloc[i]==1:
            target.iloc[i]=-1
        else:
            target.iloc[i]=1
    return target

### Matrice de confusion

In [102]:
ytest=transform_target(ytest)
confusion=metrics.confusion_matrix(ytest,ypred)
print(confusion)

[[   61     7]
 [ 7043 35448]]


### Recherche des hypermaramètres du modèle

In [103]:
parameters={
    "n_estimators":[50,100,150,200,250,300,350,400,450,500],
    "contamination":[0.1,0.15,0.175,0.2],
    "max_samples":[0.4,0.5,0.6,0.7,0.8,0.9,1.0],
    "max_features":[0.5,0.6,0.7,0.8,0.9,1.0],
    "bootstrap":[False,True]
}

In [108]:
scoring_metrics = {
    'roc_auc': make_scorer(roc_auc_score),
    'f1_score': make_scorer(f1_score),
    'accuracy_score':make_scorer(accuracy_score),
    'precision_score':make_scorer(precision_score),
    'recall_score':make_scorer(recall_score)
}

In [135]:
gridsearch=GridSearchCV(estimator=IsolationForest(random_state=42),param_grid=parameters,verbose=2,scoring=scoring_metrics,refit='roc_auc')
gridsearch

GridSearchCV(estimator=IsolationForest(random_state=42),
             param_grid={'bootstrap': [False, True],
                         'contamination': [0.1, 0.15, 0.175, 0.2],
                         'max_features': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                         'max_samples': [0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
                         'n_estimators': [50, 100, 150, 200, 250, 300, 350, 400,
                                          450, 500]},
             refit='roc_auc',
             scoring={'accuracy_score': make_scorer(accuracy_score),
                      'f1_score': make_scorer(f1_score),
                      'precision_score': make_scorer(precision_score),
                      'recall_score': make_scorer(recall_score),
                      'roc_auc': make_scorer(roc_auc_score)},
             verbose=2)

In [136]:
ytrain=transform_target(ytrain)

In [137]:
n=len(parameters['n_estimators']) * len(parameters['contamination']) * len(parameters['bootstrap']) * len(parameters['max_samples']) * len(parameters['max_features'])
n

3360

In [ ]:
gridsearch.fit(xtrain, ytrain)

Fitting 5 folds for each of 3360 candidates, totalling 16800 fits
[CV] END bootstrap=False, contamination=0.1, max_features=0.5, max_samples=0.4, n_estimators=50; total time=   1.2s
[CV] END bootstrap=False, contamination=0.1, max_features=0.5, max_samples=0.4, n_estimators=50; total time=   1.3s
[CV] END bootstrap=False, contamination=0.1, max_features=0.5, max_samples=0.4, n_estimators=50; total time=   1.3s
[CV] END bootstrap=False, contamination=0.1, max_features=0.5, max_samples=0.4, n_estimators=50; total time=   1.3s
[CV] END bootstrap=False, contamination=0.1, max_features=0.5, max_samples=0.4, n_estimators=50; total time=   1.3s
[CV] END bootstrap=False, contamination=0.1, max_features=0.5, max_samples=0.4, n_estimators=100; total time=   2.6s
[CV] END bootstrap=False, contamination=0.1, max_features=0.5, max_samples=0.4, n_estimators=100; total time=   2.5s
